In [1]:
import set_paths

In [2]:
from itertools import product
from pandas import DataFrame

In [3]:
from probability.discrete import Discrete, Conditional

# Bayesian Reasoning and Machine Learning

## 1.1 Probability Refresher 

### 1.1.1 Interpreting Conditional Probability

In [4]:
darts = Discrete.from_probs(
    data={i: 1 / 20 for i in range(1, 21)},
    variables='region'
)

In [5]:
darts.data

region
1     0.05
2     0.05
3     0.05
4     0.05
5     0.05
6     0.05
7     0.05
8     0.05
9     0.05
10    0.05
11    0.05
12    0.05
13    0.05
14    0.05
15    0.05
16    0.05
17    0.05
18    0.05
19    0.05
20    0.05
Name: p(region), dtype: float64

In [6]:
1 / 19

0.05263157894736842

In [7]:
darts.given(region__ne=20).p(region=5)

0.05263157894736841

In [8]:
darts.p(region=5, region__ne=20) / darts.p(region__ne=20)

0.05263157894736841

In [9]:
darts.p(region=5) / darts.p(region__ne=20)

0.05263157894736841

### 1.1.2 Probability Tables

In [10]:
country = Discrete.from_counts({
    'england': 60_776_238,
    'scotland': 5_116_900,
    'wales': 2_980_700
}, 'country')

In [11]:
country.data

country
england     0.882429
scotland    0.074294
wales       0.043278
Name: p(country), dtype: float64

In [12]:
language__given__country = Conditional.from_probs(
    data={
        ('english', 'england'): 0.95,
        ('english', 'scotland'): 0.7,
        ('english', 'wales'): 0.6,
        ('scottish', 'england'): 0.04,
        ('scottish', 'scotland'): 0.3,
        ('scottish', 'wales'): 0.0,
        ('welsh', 'england'): 0.01,
        ('welsh', 'scotland'): 0.0,
        ('welsh', 'wales'): 0.4,
    }, 
    joint_variables='language',
    conditional_variables='country'
)

In [13]:
language__given__country.data

country,england,scotland,wales
language,,,
english,0.95,0.7,0.6
scottish,0.04,0.3,0.0
welsh,0.01,0.0,0.4


In [14]:
language__country = language__given__country * country

In [15]:
language__country.data

country   language
england   english     0.838307
          scottish    0.035297
          welsh       0.008824
scotland  english     0.052006
          scottish    0.022288
          welsh       0.000000
wales     english     0.025967
          scottish    0.000000
          welsh       0.017311
Name: p(country,language), dtype: float64

## 1.2 Probabilistic Reasoning

#### Example 1.2

In [16]:
has_kj = Discrete.from_probs(data={
    'yes': 1e-5, 
    'no': 1 - 1e-5
}, variables='has_kj')

In [17]:
has_kj.data

has_kj
yes    0.00001
no     0.99999
Name: p(has_kj), dtype: float64

In [18]:
eats_hbs__given__has_kj = Conditional.from_probs({
        ('yes', 'yes'): 0.9,
        ('no', 'yes'): 0.1
    }, 
    joint_variables='eats_hbs', 
    conditional_variables='has_kj'
)

In [19]:
eats_hbs__given__has_kj.data

has_kj,yes
eats_hbs,
no,0.1
yes,0.9


###### 1)

In [20]:
eats_hbs = Discrete.from_probs({'yes': 0.5, 'no': 0.5}, variables='eats_hbs')

In [21]:
eats_hbs.data

eats_hbs
yes    0.5
no     0.5
Name: p(eats_hbs), dtype: float64

In [22]:
has_kj__given__eats_hbs = eats_hbs__given__has_kj * has_kj / eats_hbs

In [23]:
has_kj__given__eats_hbs.data

has_kj  eats_hbs
no      no               NaN
        yes              NaN
yes     no          0.000002
        yes         0.000018
Name: p(has_kj,eats_hbs), dtype: float64

In [24]:
has_kj__given__eats_hbs.p(has_kj='yes', eats_hbs='yes')

1.8e-05

###### 2)

In [25]:
eats_hbs = Discrete.from_probs({'yes': 0.001, 'no': 0.999}, variables='eats_hbs')

In [26]:
eats_hbs.data

eats_hbs
yes    0.001
no     0.999
Name: p(eats_hbs), dtype: float64

In [27]:
has_kj__given__eats_hbs = eats_hbs__given__has_kj * has_kj / eats_hbs

In [28]:
has_kj__given__eats_hbs.data

has_kj  eats_hbs
no      no               NaN
        yes              NaN
yes     no          0.000001
        yes         0.009000
Name: p(has_kj,eats_hbs), dtype: float64

In [29]:
has_kj__given__eats_hbs.p(has_kj='yes', eats_hbs='yes')

0.009

#### Example 1.3

In [30]:
butler = Discrete.from_probs({'yes': 0.6, 'no': 0.4}, variables='butler')

In [31]:
maid = Discrete.from_probs({'yes': 0.2, 'no': 0.8}, variables='maid')

In [32]:
butler__and__maid = butler * maid

In [33]:
butler__and__maid.data

butler  maid
yes     yes     0.12
        no      0.48
no      yes     0.08
        no      0.32
Name: p(butler,maid), dtype: float64

In [34]:
knife__given__butler__and__maid = Conditional.from_probs(data={
        ('yes', 'no', 'no'): 0.3,
        ('yes', 'no', 'yes'): 0.2,
        ('yes', 'yes', 'no'): 0.6,
        ('yes', 'yes', 'yes'): 0.1,
        ('no', 'no', 'no'): 0.7,
        ('no', 'no', 'yes'): 0.8,
        ('no', 'yes', 'no'): 0.4,
        ('no', 'yes', 'yes'): 0.9,
    }, 
    joint_variables='knife_used', 
    conditional_variables=['butler', 'maid']
)

In [35]:
knife__given__butler__and__maid.data

butler       no       yes     
maid         no  yes   no  yes
knife_used                    
no          0.7  0.8  0.4  0.9
yes         0.3  0.2  0.6  0.1

In [36]:
butler__and__maid__and__knife = knife__given__butler__and__maid * butler__and__maid

In [37]:
butler__and__maid__and__knife.data

butler  maid  knife_used
no      no    no            0.224
              yes           0.096
        yes   no            0.064
              yes           0.016
yes     no    no            0.192
              yes           0.288
        yes   no            0.108
              yes           0.012
Name: p(butler,maid,knife_used), dtype: float64

In [38]:
butler__given__knife = butler__and__maid__and__knife.given(knife_used='yes').p(butler='yes')

In [39]:
butler__given__knife

0.7281553398058251

#### Example 1.4

In [40]:
occupied__given__alice__and__bob = Conditional.binary_from_probs({
        (False, False): 1,
        (False, True): 1,
        (True, False): 1,
        (True, True): 0,
    },
    joint_variable='occupied',
    conditional_variables=['alice', 'bob']
)

In [41]:
occupied__given__alice__and__bob.data

alice    False       True       
bob      False True  False True 
occupied                        
0            0     0     0     1
1            1     1     1     0

In [42]:
alice__and__bob = Discrete.from_probs({
    (False, False): 0.25,
    (False, True): 0.25,
    (True, False): 0.25,
    (True, True): 0.25,
}, variables=['alice', 'bob'])

In [43]:
alice__and__bob.data

alice  bob  
False  False    0.25
       True     0.25
True   False    0.25
       True     0.25
Name: p(alice,bob), dtype: float64

In [44]:
alice__and__bob__and__occupied = occupied__given__alice__and__bob * alice__and__bob

In [45]:
alice__and__bob__and__occupied.given(alice=True, occupied=True).p(bob=False)

1.0

In [46]:
occupied__given__alice__and__bob = Conditional.binary_from_probs({
        (False, False): 1,
        (False, True): 1,
        (True, False): 1,
        (True, True): 0,
    },
    joint_variable='occupied',
    conditional_variables=['alice', 'bob']
)

In [47]:
occupied__given__alice__and__bob.data

alice    False       True       
bob      False True  False True 
occupied                        
0            0     0     0     1
1            1     1     1     0

In [48]:
alice__and__bob = Discrete.from_probs({
    (False, False): 0.25,
    (False, True): 0.25,
    (True, False): 0.25,
    (True, True): 0.25,
}, variables=['alice', 'bob'])

In [49]:
alice__and__bob.data

alice  bob  
False  False    0.25
       True     0.25
True   False    0.25
       True     0.25
Name: p(alice,bob), dtype: float64

In [50]:
alice__and__bob__and__occupied = occupied__given__alice__and__bob * alice__and__bob

In [51]:
alice__and__bob__and__occupied.given(alice=True, occupied=True).p(bob=False)

1.0

#### Example 1.7

##### xor

In [52]:
xor = Conditional.from_probs(
    data={
        (1, 0, 0): 0,
        (1, 0, 1): 1,
        (1, 1, 0): 1,
        (1, 1, 1): 0,
    },
    joint_variables='A_xor_B',
    conditional_variables=['A', 'B']
)

In [53]:
xor.data

A        0     1   
B        0  1  0  1
A_xor_B            
1        0  1  1  0

##### soft xor

In [54]:
c__given__a__and__b = Conditional.binary_from_probs(
    data={
        (0, 0): 0.1,
        (0, 1): 0.99,
        (1, 0): 0.8,
        (1, 1): 0.25,
    },
    joint_variable='C',
    conditional_variables=['A', 'B']
)

In [55]:
c__given__a__and__b.data

A    0          1      
B    0     1    0     1
C                      
0  0.9  0.01  0.2  0.75
1  0.1  0.99  0.8  0.25

In [56]:
a = Discrete.binary(0.65, 'A')

In [57]:
a.data

A
0    0.35
1    0.65
Name: p(A), dtype: float64

In [58]:
b = Discrete.binary(0.77, 'B')

In [59]:
b.data

B
0    0.23
1    0.77
Name: p(B), dtype: float64

In [60]:
a__and__b = a * b

In [61]:
a__and__b.data

A  B
0  0    0.0805
   1    0.2695
1  0    0.1495
   1    0.5005
Name: p(A,B), dtype: float64

In [62]:
a__and__b__and__c = a__and__b * c__given__a__and__b

In [63]:
a__and__b__and__c.data

A  B  C
0  0  0    0.072450
      1    0.008050
   1  0    0.002695
      1    0.266805
1  0  0    0.029900
      1    0.119600
   1  0    0.375375
      1    0.125125
Name: p(A,B,C), dtype: float64

In [64]:
a__and__b__and__c.given(C=0).p(A=1)

0.8435847799841806

### 1.3.1 Two dice : what were the individual scores?

In [74]:
t = Discrete.from_observations(
    data = DataFrame({
        't': [s_a + s_b 
              for s_a, s_b in product(range(1, 7), range(1, 7))]
    })
)

In [75]:
t.data

t
2     0.027778
3     0.055556
4     0.083333
5     0.111111
6     0.138889
7     0.166667
8     0.138889
9     0.111111
10    0.083333
11    0.055556
12    0.027778
Name: p(t), dtype: float64

In [76]:
s_a__s_b = Discrete.from_probs(
    data = {
        (a, b): 1 / 36 
        for a, b in product(range(1, 7), range(1, 7))
    },
    variables=['s_a', 's_b']
)

In [77]:
s_a__s_b.data.unstack('s_a')

s_a,1,2,3,4,5,6
s_b,,,,,,
1,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
2,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
3,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
4,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
5,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778
6,0.027778,0.027778,0.027778,0.027778,0.027778,0.027778


In [82]:
t_9__given__s_a__s_b = Conditional.from_probs(
    data={
        (9, a, b): int(a + b == 9)
        for a, b in product(range(1, 7), range(1, 7))
    },
    joint_variables=['t'],
    conditional_variables=['s_a', 's_b']
)

In [83]:
t_9__given__s_a__s_b.data.stack('s_b')

s_a    1  2  3  4  5  6
t s_b                  
9 1    0  0  0  0  0  0
  2    0  0  0  0  0  0
  3    0  0  0  0  0  1
  4    0  0  0  0  1  0
  5    0  0  0  1  0  0
  6    0  0  1  0  0  0

In [86]:
t_9__s_a__s_b = t_9__given__s_a__s_b * s_a__s_b

In [87]:
t_9__s_a__s_b.data.unstack('s_a')

,s_a,1,2,3,4,5,6
s_b,t,,,,,,
1,9,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,9,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,9,0.0,0.0,0.000000,0.000000,0.000000,0.027778
4,9,0.0,0.0,0.000000,0.000000,0.027778,0.000000
5,9,0.0,0.0,0.000000,0.027778,0.000000,0.000000
6,9,0.0,0.0,0.027778,0.000000,0.000000,0.000000


In [90]:
t_9 = t_9__s_a__s_b / t.p(t=9)

In [92]:
t_9.data.unstack('s_a')

,s_a,1,2,3,4,5,6
s_b,t,,,,,,
1,9,0.0,0.0,0.00,0.00,0.00,0.00
2,9,0.0,0.0,0.00,0.00,0.00,0.00
3,9,0.0,0.0,0.00,0.00,0.00,0.25
4,9,0.0,0.0,0.00,0.00,0.25,0.00
5,9,0.0,0.0,0.00,0.25,0.00,0.00
6,9,0.0,0.0,0.25,0.00,0.00,0.00
